# Coding out the Solow Growth Model
## A graphical analysis

Below we import libraries that we need to use.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

### Use case 1: use Python just as how you would use a calculator
Let's replicate our homework results. Calculate the steady state:
1. Let our production function be: $Y = F(K, L) = K^\alpha + L^{1-\alpha}$. 
2. Looking at the per capita version, we have $f(k) = k^\alpha$.
3. From our homework we know that the steady-state $k^*$ is given by: $$k^* = \left(\frac{s}{\delta}\right)^{\frac{1}{1-\alpha}}$$

So let's make a calculator (function).

In [ ]:
# define parameter values
delta = 0.05
alpha = 0.5

# find steady states first
def steady_state(s):
    k_star = (s / delta)**(1/(1 - alpha))
    return k_star

# initiate our target saving rates
s_list = [0.1, 0.5, 0.9]

# a blank list to store data
k_star_list = []

for s in s_list:
    k_star = steady_state(s)
    y_star = k_star**alpha
    print(rf"When saving rate is {s}, k* is {k_star} and y* is {y_star}")

And if you recall, these were the numbers in homework 1.

### Use case 2: Use it to visulise dynamics

#### Law of motion
Recall that the law of motion of $k$ in discrete time is given by:
$$k_{t+1} = sf(k_t) + (1 - \delta) k_t$$

In [ ]:
s = 0.3
s1 = 0.5
alpha = 0.3
delta = 0.4

kmin, kmax = 0, 2
kgrid = np.linspace(kmin, kmax, 200)

def lom(s=s, k=kgrid):
    return s * k**alpha + (1 - delta) * k

# create dataframe
lom_values = lom()
lom_values_2 = lom(s=s1)

dic = {'x': kgrid, 'lom_values': lom_values,
       'lom_values_2': lom_values_2, 'diagonal': kgrid}
df = pd.DataFrame(dic)

k_star = steady_state(s)
k_star_2 = steady_state(s1)

lb = r'$k_{t+1} = g(k_t) = sk_t^{\alpha} + (1 - \delta)k_t$'
lb_2 = r'$k_{t+1} = g_1(k_t) = s_1k_t^{\alpha} + (1 - \delta)k_t$'

# initiate plot object
fig, ax = plt.subplots()

# plot lines and mark steadt state
ax.plot('x', 'lom_values', 'g-', data=df, lw=2, alpha=0.6, label=lb)
ax.plot('x', 'lom_values_2', 'b-', data=df, lw=2, alpha=0.6, label=lb_2)
ax.plot('x', 'diagonal', 'k-', data=df, lw=1,
        alpha=0.7, label='45 Degree Line')
ax.plot(k_star, k_star, 'go', ms=10, alpha=0.6)
ax.plot(k_star_2, k_star_2, 'bo', ms=10, alpha=0.6)

# annotate the chart
annotation_args = dict(xytext=(-40, -60),
                       textcoords='offset points',
                       fontsize=12,
                       arrowprops=dict(arrowstyle="->"))

ax.annotate(rf"{r'$k^*_{old}=$'}{round(k_star,3)}",
            xy=(k_star, k_star), **annotation_args)

ax.annotate(rf"{r'$k^*_{new}=$'}{round(k_star_2,3)}",
            xy=(k_star_2, k_star_2), **annotation_args)

ax.set_xlim(kmin, kmax)
ax.set_ylim(kmin, kmax)
ax.legend(loc='upper left', frameon=False, fontsize=12)

ax.set_xlabel('$k_t$', fontsize=12)
ax.set_ylabel('$k_{t+1}$', fontsize=12)

plt.show()


#### Trajectories

We know that it doesn't matter where we start, we will converge to $k^*$. It is often hard to draw by hand the trajectories but it is fairly easy using Python.

In [ ]:
def plot_trajectory(k0, num_arrows=5):
    
    fig, ax = plt.subplots()
    ax.set_xlim(kmin, kmax)
    ax.set_ylim(kmin, kmax)

    lom_values = lom()
    
    # formatting for arrows
    hw = (kmax - kmin) * 0.01
    hl = 2 * hw
    arrow_args = dict(fc="k", ec="k", head_width=hw,
                    length_includes_head=True, lw=1, alpha=0.6, head_length=hl)
    lb = r'$k_{t+1} = g(k_t)$'

    k_star = steady_state(s)
    ax.plot(k_star, k_star, 'go', ms=10, alpha=0.6)

    # plotting out LOM 
    ax.plot(kgrid, lom_values, 'g-', lw=2, alpha=0.6, label=lb)
    ax.plot(kgrid, kgrid, 'k-', lw=1, alpha=0.7, label='45 Degree Line')
    
    k = k0
    xticks = [kmin]
    xtick_labels = [kmin]

    for i in range(num_arrows): 
        if i == 0:
            ax.arrow(k, 0.0, 0.0, lom(k=k), **arrow_args) # x, y, dx, dy 
        else:
            ax.arrow(k, k, 0.0,lom(k=k) - k, **arrow_args)
            ax.plot((k, k), (0, k), 'k', ls='dotted')
        
        ax.arrow(k, lom(k=k), lom(k=k) - k, 0, **arrow_args)

        xticks.append(k)
        xtick_labels.append(r'${}_{}$'.format("k", str(i)))
        
        k = lom(k=k)
        xticks.append(k)
        xtick_labels.append(r'${}_{}$'.format("k", str(i+1)))
        ax.plot((k, k), (0, k), 'k', ls='dotted')

    xticks.append(kmax)
    xtick_labels.append(kmax)
    ax.set_xticks(xticks)
    ax.set_yticks(xticks)
    ax.set_xticklabels(xtick_labels)
    ax.set_yticklabels(xtick_labels)
    ax.legend(loc='upper left', frameon=False, fontsize=12)
    plt.title(rf"{r'$k_0$ = '}{round(k0,3)}")
    plt.show()

plot_trajectory(0.1)

In [ ]:
plot_trajectory(1.5)

#### Time path in the context of comparative static

I claim that you can write some simple codes to simulate the time path of $k$ in response to a change in $s$.

In [ ]:
def solow_model(k0, s_old, s_new, threshold=50, T=300):
    
    k = k0
    y = k0**alpha
    k_dot =  lom(s=s_old, k=k) - k

    s_list = [s_old]
    k_list = [k]
    y_list = [y]
    k_dot_list = [k_dot]
    
    for i in range(T):
        
        if i < threshold:
           
           s = s_old
        
        else:
            s = s_new
            
        k_dot = lom(s=s, k=k) - k
        k = k + k_dot
        y = k**alpha

        s_list.append(s)
        k_list.append(k)
        y_list.append(y)
        k_dot_list.append(k_dot)
        
    dic = {'saving rate':s_list, 'change in k':k_dot_list, 'capital per capita':k_list, 'output per capita':y_list}
    df = pd.DataFrame(dic)
    
    return df       

In [ ]:
k_star_old = steady_state(s)
df = solow_model(k0=k_star_old, s_old=s, s_new=s1)
df.plot(subplots=True, title=['Time path after a change in saving rate', '', '', ''])
#plt.savefig("plot.png")